# Pandas en practica: Tipos, Missings, Memoria y Pipelines

[![Open in Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/sonder-art/fdd_p26/blob/main/clase/12_pandas/code/01_limpieza_de_datos.ipynb)

Este notebook cubre la parte hands-on del modulo. Vamos a trabajar con un dataset generado que tiene todos los problemas tipicos de datos reales: tipos mal inferidos, missings en distintos formatos, columnas pesadas, y codigo lento.

**Tiempo estimado**: ~35 minutos

In [1]:
import pandas as pd
import numpy as np

print(f"pandas: {pd.__version__}")
print(f"numpy:  {np.__version__}")

pandas: 2.0.3
numpy:  1.24.3


---
## 1. Generar dataset sucio

Vamos a crear un DataFrame que simula datos reales con problemas reales: missings, tipos inconsistentes, strings sucios.

In [2]:
np.random.seed(42)
n = 100_000

df_sucio = pd.DataFrame({
    # ID: entero, pero vamos a meter missings para forzar float
    "id_cliente": np.where(
        np.random.random(n) < 0.02,
        np.nan,
        np.arange(1, n + 1).astype(float),
    ),
    # Nombre: strings con variaciones de missing
    "nombre": np.random.choice(
        ["Ana Lopez", "Carlos Ruiz", "  maria garcia ", "PEDRO SANCHEZ",
         "", "N/A", "null", "na", np.nan, "Sofia Torres"],
        size=n,
    ),
    # Genero: categorico con missings disfrazados
    "genero": np.random.choice(
        ["M", "F", "NB", "N/A", "", np.nan, "No especificado"],
        size=n,
        p=[0.35, 0.35, 0.05, 0.05, 0.05, 0.10, 0.05],
    ),
    # Edad: numerico con missings y outliers
    "edad": np.where(
        np.random.random(n) < 0.08,
        np.nan,
        np.clip(np.random.normal(35, 12, n).astype(int), -5, 200),
    ),
    # Monto: float con algunos negativos (errores) y missings
    "monto": np.where(
        np.random.random(n) < 0.05,
        np.nan,
        np.round(np.random.exponential(500, n) * np.random.choice([1, 1, 1, -1], n), 2),
    ),
    # Fecha: string con formatos mixtos
    "fecha": np.random.choice(
        ["2024-01-15", "15/02/2024", "2024-03-20", "20-04-2024",
         "2024-05-10", "", np.nan, "not_a_date"],
        size=n,
    ),
    # Activo: booleano con missings
    "activo": np.random.choice(
        [True, False, np.nan],
        size=n,
        p=[0.6, 0.3, 0.1],
    ),
    # Region: categorico con muchas repeticiones (candidato a category)
    "region": np.random.choice(
        ["Norte", "Sur", "Este", "Oeste", "Centro",
         "norte", "NORTE", "Sur ", " este"],
        size=n,
    ),
})

print(f"Shape: {df_sucio.shape}")
df_sucio.head(10)

Shape: (100000, 8)


,id_cliente,nombre,genero,edad,monto,fecha,activo,region
0,1.0,Carlos Ruiz,,53.0,856.40,not_a_date,0.0,este
1,2.0,na,M,47.0,301.38,2024-01-15,0.0,NORTE
2,3.0,nan,F,45.0,-720.19,15/02/2024,1.0,Centro
3,4.0,Ana Lopez,F,59.0,213.96,2024-05-10,1.0,Sur
4,5.0,N/A,nan,28.0,787.17,,1.0,Centro
5,6.0,N/A,F,47.0,1363.65,2024-01-15,1.0,Norte
6,7.0,null,M,59.0,19.05,not_a_date,1.0,este
7,8.0,N/A,M,33.0,951.28,nan,1.0,este
8,9.0,Ana Lopez,M,52.0,-1459.81,not_a_date,1.0,Oeste
9,10.0,N/A,F,39.0,199.45,2024-05-10,NaN,Sur


---
## 2. Diagnostico inicial

Antes de tocar nada, diagnostica. Esto deberia ser lo **primero** que haces con cualquier dataset.

In [3]:
# info() es tu mejor amigo: tipos, nulls, y memoria en un solo comando
df_sucio.info(memory_usage="deep")

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100000 entries, 0 to 99999
Data columns (total 8 columns):
 #   Column      Non-Null Count   Dtype  
---  ------      --------------   -----  
 0   id_cliente  98023 non-null   float64
 1   nombre      100000 non-null  object 
 2   genero      100000 non-null  object 
 3   edad        91895 non-null   float64
 4   monto       95026 non-null   float64
 5   fecha       100000 non-null  object 
 6   activo      90032 non-null   float64
 7   region      100000 non-null  object 
dtypes: float64(4), object(4)
memory usage: 26.9 MB


In [4]:
# Porcentaje de missings por columna
print("% missings por columna:")
print(df_sucio.isna().mean().round(4) * 100)

% missings por columna:
id_cliente    1.98
nombre        0.00
genero        0.00
edad          8.10
monto         4.97
fecha         0.00
activo        9.97
region        0.00
dtype: float64


In [5]:
# Tipos de datos — nota como pandas eligio los tipos
print("Tipos:")
print(df_sucio.dtypes)
print()
print("Observaciones:")
print("- id_cliente es float64 (deberia ser entero, pero NaN lo fuerza a float)")
print("- nombre, genero, fecha, region son object (strings mezclados con NaN)")
print("- activo es object (deberia ser bool, pero NaN lo corrompe)")

Tipos:
id_cliente    float64
nombre         object
genero         object
edad          float64
monto         float64
fecha          object
activo        float64
region         object
dtype: object

Observaciones:
- id_cliente es float64 (deberia ser entero, pero NaN lo fuerza a float)
- nombre, genero, fecha, region son object (strings mezclados con NaN)
- activo es object (deberia ser bool, pero NaN lo corrompe)


---
## 3. El problema de `np.nan`

`np.nan` es un `float` de IEEE 754. Esta decision tecnica tiene consecuencias reales para tu analisis.

In [6]:
# np.nan es float
print(f"type(np.nan) = {type(np.nan)}")
print(f"np.nan == np.nan = {np.nan == np.nan}")  # False! IEEE 754
print()

# Consecuencia 1: enteros con NaN se promueven a float
s_int = pd.Series([1, 2, 3, np.nan, 5])
print(f"Serie con NaN: dtype = {s_int.dtype}")  # float64, no int
print(f"Valores: {s_int.tolist()}")  # 1.0, 2.0, 3.0, nan, 5.0
print()

# Consecuencia 2: booleanos con NaN se vuelven object
s_bool = pd.Series([True, False, np.nan])
print(f"Serie bool con NaN: dtype = {s_bool.dtype}")  # object!
print()

# Consecuencia 3: NaN rompe comparaciones
print(f"np.nan in [np.nan] = {np.nan in [np.nan]}")  # True (Python 'is')
datos = pd.Series([1, np.nan, 2, np.nan])
print(f"value_counts():\n{datos.value_counts()}")  # NaN excluido por default
print(f"value_counts(dropna=False):\n{datos.value_counts(dropna=False)}")

type(np.nan) = <class 'float'>
np.nan == np.nan = False

Serie con NaN: dtype = float64
Valores: [1.0, 2.0, 3.0, nan, 5.0]

Serie bool con NaN: dtype = object

np.nan in [np.nan] = True
value_counts():
1.0    1
2.0    1
Name: count, dtype: int64
value_counts(dropna=False):
NaN    2
1.0    1
2.0    1
Name: count, dtype: int64


In [7]:
# La solucion: nullable dtypes con pd.NA
s_nullable = pd.array([1, 2, 3, pd.NA, 5], dtype="Int64")  # I mayuscula!
print(f"Nullable Int64: dtype = {pd.Series(s_nullable).dtype}")
print(f"Valores: {list(s_nullable)}")  # 1, 2, 3, <NA>, 5 — enteros reales
print()

# pd.NA vs np.nan: propagacion consistente
print(f"pd.NA + 1 = {pd.NA + 1}")     # <NA> — propaga
print(f"pd.NA | True = {pd.NA | True}")  # True — logica de 3 valores
print(f"pd.NA & False = {pd.NA & False}")  # False

Nullable Int64: dtype = Int64
Valores: [1, 2, 3, <NA>, 5]

pd.NA + 1 = <NA>
pd.NA | True = True
pd.NA & False = False


---
## 4. Missings por tipo de dato

Los missings se comportan distinto segun el tipo de columna. Vamos caso por caso.

### 4.1 Missings numericos

In [8]:
# El peligro de fillna(0)
edad = df_sucio["edad"].copy()

print(f"Media original (ignorando NaN): {edad.mean():.2f}")
print(f"N original: {edad.notna().sum()}")
print()

# fillna(0): introduce sesgo si 0 no es un valor valido
edad_fill0 = edad.fillna(0)
print(f"Media con fillna(0): {edad_fill0.mean():.2f}  ← sesgo hacia abajo")

# fillna(mediana): mejor para distribucion simetrica
edad_fillmed = edad.fillna(edad.median())
print(f"Media con fillna(median): {edad_fillmed.mean():.2f}  ← preserva mejor")

# dropna: reduce N pero no introduce sesgo
print(f"Media con dropna: {edad.dropna().mean():.2f}, N={edad.dropna().shape[0]}")
print()

# Impacto en varianza
print(f"Std original: {edad.std():.2f}")
print(f"Std fillna(0): {edad_fill0.std():.2f}  ← inflada por los ceros")
print(f"Std fillna(median): {edad_fillmed.std():.2f}  ← reducida (comprime hacia centro)")

Media original (ignorando NaN): 34.52
N original: 91895

Media con fillna(0): 31.72  ← sesgo hacia abajo
Media con fillna(median): 34.48  ← preserva mejor
Media con dropna: 34.52, N=91895

Std original: 12.03
Std fillna(0): 14.89  ← inflada por los ceros
Std fillna(median): 11.54  ← reducida (comprime hacia centro)


In [9]:
# Nullable Int64 vs float64
id_float = df_sucio["id_cliente"]
id_nullable = df_sucio["id_cliente"].astype("Int64")

print(f"Tipo original: {id_float.dtype} — IDs son 1.0, 2.0, 3.0...")
print(f"Tipo nullable: {id_nullable.dtype} — IDs son 1, 2, 3, <NA>")
print()
print("Primeros valores con missing:")
mask = id_float.isna()
idx = mask[mask].index[0]
print(f"  float64: {id_float.iloc[idx-1:idx+2].tolist()}")
print(f"  Int64:   {id_nullable.iloc[idx-1:idx+2].tolist()}")

Tipo original: float64 — IDs son 1.0, 2.0, 3.0...
Tipo nullable: Int64 — IDs son 1, 2, 3, <NA>

Primeros valores con missing:
  float64: [72.0, nan, 74.0]
  Int64:   [72, <NA>, 74]


### 4.2 Missings en texto y categoricos

En columnas de texto, los missings vienen disfrazados: `NaN`, `""`, `"N/A"`, `"null"`, `"na"`, espacios...

In [10]:
# ¿Cuantos "missings reales" hay en genero?
print("value_counts de genero (raw):")
print(df_sucio["genero"].value_counts(dropna=False))
print()
print('NaN, "", "N/A", y "No especificado" son todos el mismo concepto:')
print("  datos faltantes. Pero pandas no lo sabe.")

value_counts de genero (raw):
genero
F                  35024
M                  34975
nan                10109
NB                  5047
No especificado     4990
N/A                 4986
                    4869
Name: count, dtype: int64

NaN, "", "N/A", y "No especificado" son todos el mismo concepto:
  datos faltantes. Pero pandas no lo sabe.


In [11]:
# Patron: normalizar missings disfrazados
def normalizar_missings_texto(series, extras=None):
    """Reemplaza variaciones de missing en strings por pd.NA."""
    valores_missing = {"", "N/A", "n/a", "na", "NA", "null", "NULL",
                       "None", "none", ".", "-", "No especificado"}
    if extras:
        valores_missing.update(extras)
    resultado = series.copy()
    resultado = resultado.str.strip()  # quitar espacios
    resultado = resultado.replace(valores_missing, pd.NA)
    return resultado

genero_limpio = normalizar_missings_texto(df_sucio["genero"])
print("Despues de normalizar:")
print(genero_limpio.value_counts(dropna=False))

Despues de normalizar:
genero
F       35024
M       34975
<NA>    14845
nan     10109
NB       5047
Name: count, dtype: int64


In [12]:
# category dtype: ahorra memoria cuando hay pocos valores unicos
print(f"object:   {df_sucio['region'].memory_usage(deep=True):>10,} bytes")

region_cat = df_sucio["region"].astype("category")
print(f"category: {region_cat.memory_usage(deep=True):>10,} bytes")
print(f"Ahorro:   {1 - region_cat.memory_usage(deep=True) / df_sucio['region'].memory_usage(deep=True):.1%}")

object:    6,166,278 bytes
category:    100,983 bytes
Ahorro:   98.4%


### 4.3 Missings en fechas y booleanos

In [13]:
# Fechas: NaT es el missing nativo de datetime64
fechas = pd.to_datetime(df_sucio["fecha"], errors="coerce", dayfirst=False)
print(f"Tipo: {fechas.dtype}")
print(f"Missings (NaT): {fechas.isna().sum()} ({fechas.isna().mean():.1%})")
print(f"  — Incluye NaN originales + strings invalidos como 'not_a_date'")
print()

# Booleanos: pd.BooleanDtype
activo_raw = df_sucio["activo"]
print(f"Tipo raw: {activo_raw.dtype}")  # object!

activo_nullable = activo_raw.astype("boolean")
print(f"Tipo nullable: {activo_nullable.dtype}")  # boolean (nullable)
print(f"Valores unicos: {activo_nullable.unique().tolist()}")

Tipo: datetime64[ns]
Missings (NaT): 37562 (37.6%)
  — Incluye NaN originales + strings invalidos como 'not_a_date'

Tipo raw: float64
Tipo nullable: boolean
Valores unicos: [False, True, <NA>]


/tmp/ipykernel_164188/2182009145.py:2: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  fechas = pd.to_datetime(df_sucio["fecha"], errors="coerce", dayfirst=False)


### 4.4 Impacto estadistico de missings

El problema mas sutil: tu `n` cambia silenciosamente entre operaciones.

In [14]:
# Ejemplo: groupby con distintos missings por grupo
ejemplo = pd.DataFrame({
    "grupo": ["A", "A", "A", "A", "B", "B", "B", "B"],
    "valor": [10, 20, np.nan, np.nan, 10, 20, 30, 40],
})

print("Datos:")
print(ejemplo)
print()

g = ejemplo.groupby("grupo")["valor"]
print("mean() por grupo:")
print(g.mean())
print()
print("count() por grupo:")
print(g.count())
print()
print("Grupo A: mean=15.0 basado en n=2 (50% missing)")
print("Grupo B: mean=25.0 basado en n=4 (0% missing)")
print("Comparar estas medias directamente es estadisticamente dudoso.")

Datos:
  grupo  valor
0     A   10.0
1     A   20.0
2     A    NaN
3     A    NaN
4     B   10.0
5     B   20.0
6     B   30.0
7     B   40.0

mean() por grupo:
grupo
A    15.0
B    25.0
Name: valor, dtype: float64

count() por grupo:
grupo
A    2
B    4
Name: valor, dtype: int64

Grupo A: mean=15.0 basado en n=2 (50% missing)
Grupo B: mean=25.0 basado en n=4 (0% missing)
Comparar estas medias directamente es estadisticamente dudoso.


---
## 5. Memoria: diagnóstico y optimización

Antes de “optimizar”, conviene **medir**: cuánta memoria usa tu DataFrame en total y qué columnas son las más pesadas. Esto suele estar dominado por columnas tipo texto (`object`) y por tipos numéricos más grandes de lo necesario (por ejemplo `float64` cuando `float32` basta).

In [15]:
# Memoria actual del dataset sucio
mem_antes = df_sucio.memory_usage(deep=True)
total_antes = mem_antes.sum()
print("Memoria por columna (MB):")
print((mem_antes / 1024**2).round(2))
print(f"\nTotal: {total_antes / 1024**2:.2f} MB")

Memoria por columna (MB):
Index         0.00
id_cliente    0.76
nombre        6.12
genero        5.63
edad          0.76
monto         0.76
fecha         6.19
activo        0.76
region        5.88
dtype: float64

Total: 26.87 MB


In [16]:
# Optimizar tipos
df_opt = df_sucio.copy()

# 1. IDs: float64 → Int64 (nullable)
df_opt["id_cliente"] = df_opt["id_cliente"].astype("Int64")

# 2. Edad: float64 → Int16 (nullable, rango -32k a 32k, sobra para edad)
df_opt["edad"] = df_opt["edad"].astype("Int16")

# 3. Monto: float64 → float32 (precision suficiente para montos)
df_opt["monto"] = df_opt["monto"].astype("float32")

# 4. Strings con pocos unicos → category
df_opt["genero"] = df_opt["genero"].astype("category")
df_opt["region"] = df_opt["region"].astype("category")

# 5. Activo: object → boolean (nullable)
df_opt["activo"] = df_opt["activo"].astype("boolean")

# 6. Nombre: object → StringDtype
df_opt["nombre"] = df_opt["nombre"].astype("string")

# Comparar
mem_despues = df_opt.memory_usage(deep=True)
total_despues = mem_despues.sum()

print("Comparacion de memoria (MB):")
comparacion = pd.DataFrame({
    "antes": (mem_antes / 1024**2).round(3),
    "despues": (mem_despues / 1024**2).round(3),
})
comparacion["reduccion"] = (1 - comparacion["despues"] / comparacion["antes"]).map("{:.0%}".format)
print(comparacion)
print(f"\nTotal: {total_antes/1024**2:.2f} MB → {total_despues/1024**2:.2f} MB")
print(f"Reduccion: {1 - total_despues/total_antes:.0%}")

Comparacion de memoria (MB):
            antes  despues reduccion
Index       0.000    0.000      nan%
id_cliente  0.763    0.858      -12%
nombre      6.123    6.123        0%
genero      5.627    0.096       98%
edad        0.763    0.286       63%
monto       0.763    0.381       50%
fecha       6.187    6.187        0%
activo      0.763    0.191       75%
region      5.880    0.096       98%

Total: 26.87 MB → 14.22 MB
Reduccion: 47%


---
## 6. Pipeline completo con `.pipe()`

Ahora vamos a juntar todo en un pipeline limpio y reproducible.

In [17]:
# --- Funciones del pipeline ---

def validar_schema(df, columnas):
    """Verifica que las columnas esperadas existan."""
    faltantes = set(columnas) - set(df.columns)
    assert not faltantes, f"Columnas faltantes: {faltantes}"
    assert len(df) > 0, "DataFrame vacio"
    return df


def normalizar_columnas(df):
    """Limpia nombres de columnas."""
    df = df.copy()
    df.columns = (
        df.columns
        .str.strip()
        .str.lower()
        .str.replace(r"[^a-z0-9]+", "_", regex=True)
        .str.strip("_")
    )
    return df


def castear_tipos(df):
    """Asigna tipos correctos."""
    return df.assign(
        id_cliente=lambda d: d["id_cliente"].astype("Int64"),
        edad=lambda d: d["edad"].astype("Int16"),
        monto=lambda d: d["monto"].astype("float32"),
        activo=lambda d: d["activo"].astype("boolean"),
        genero=lambda d: d["genero"].astype("string"),
        nombre=lambda d: d["nombre"].astype("string"),
        region=lambda d: d["region"].astype("string"),
    )


def limpiar_texto(df):
    """Normaliza strings y missings disfrazados."""
    valores_missing = {"N/A", "n/a", "na", "NA", "null", "NULL",
                       "None", "none", ".", "-", "No especificado"}

    def limpiar_col(series):
        s = series.str.strip().str.lower()
        s = s.replace(valores_missing | {""}, pd.NA)
        return s

    return df.assign(
        nombre=lambda d: limpiar_col(d["nombre"]).str.title(),
        genero=lambda d: limpiar_col(d["genero"]).str.upper(),
        region=lambda d: limpiar_col(d["region"]).str.capitalize(),
    )


def limpiar_fechas(df):
    """Parsea fechas, invalidas se vuelven NaT."""
    return df.assign(
        fecha=lambda d: pd.to_datetime(d["fecha"], errors="coerce", dayfirst=False)
    )


def filtrar_invalidos(df):
    """Quita filas con problemas irrecuperables."""
    return (
        df
        .query("edad >= 0 | edad != edad")  # permitir NaN pero no negativos
        .query("monto >= 0 | monto != monto")  # permitir NaN pero no negativos
    )


def convertir_categorias(df):
    """Convierte columnas con pocos unicos a category."""
    return df.assign(
        genero=lambda d: d["genero"].astype("category"),
        region=lambda d: d["region"].astype("category"),
    )


def validar_salida(df):
    """Asserts de cordura sobre el resultado."""
    assert df["id_cliente"].dropna().is_unique, "IDs duplicados"
    assert df["edad"].dropna().between(0, 120).all(), "Edades fuera de rango"
    assert df["monto"].dropna().ge(0).all(), "Montos negativos"
    return df

In [18]:
# --- Ejecutar pipeline ---

COLUMNAS_ESPERADAS = ["id_cliente", "nombre", "genero", "edad",
                      "monto", "fecha", "activo", "region"]

df_limpio = (
    df_sucio
    .pipe(validar_schema, COLUMNAS_ESPERADAS)
    .pipe(normalizar_columnas)
    .pipe(castear_tipos)
    .pipe(limpiar_texto)
    .pipe(limpiar_fechas)
    .pipe(filtrar_invalidos)
    .pipe(convertir_categorias)
    .pipe(validar_salida)
)

print(f"Filas: {len(df_sucio):,} → {len(df_limpio):,} ({len(df_sucio) - len(df_limpio):,} eliminadas)")
print()
df_limpio.info(memory_usage="deep")

Filas: 100,000 → 69,931 (30,069 eliminadas)

<class 'pandas.core.frame.DataFrame'>
Index: 69931 entries, 0 to 99997
Data columns (total 8 columns):
 #   Column      Non-Null Count  Dtype         
---  ------      --------------  -----         
 0   id_cliente  68550 non-null  Int64         
 1   nombre      41963 non-null  string        
 2   genero      63126 non-null  category      
 3   edad        69931 non-null  Int16         
 4   monto       65356 non-null  float32       
 5   fecha       43645 non-null  datetime64[ns]
 6   activo      62998 non-null  boolean       
 7   region      69931 non-null  category      
dtypes: Int16(1), Int64(1), boolean(1), category(2), datetime64[ns](1), float32(1), string(1)
memory usage: 6.2 MB


/tmp/ipykernel_164188/1481620905.py:57: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  fecha=lambda d: pd.to_datetime(d["fecha"], errors="coerce", dayfirst=False)


In [19]:
df_limpio.head(10)

,id_cliente,nombre,genero,edad,monto,fecha,activo,region
0,1,Carlos Ruiz,<NA>,53,856.400024,NaT,False,Este
1,2,<NA>,M,47,301.380005,2024-01-15,False,Norte
3,4,Ana Lopez,F,59,213.960007,2024-05-10,True,Sur
4,5,<NA>,NAN,28,787.169983,NaT,True,Centro
5,6,<NA>,F,47,1363.650024,2024-01-15,True,Norte
6,7,<NA>,M,59,19.049999,NaT,True,Este
7,8,<NA>,M,33,951.280029,NaT,True,Este
9,10,<NA>,F,39,199.449997,2024-05-10,<NA>,Sur
10,11,Pedro Sanchez,F,46,1529.199951,NaT,True,Este
12,13,Sofia Torres,<NA>,31,246.059998,2024-04-20,True,Centro


---
## 7. Formatos: CSV vs Parquet

Guardar en el formato correcto es una decision arquitectonica, no estetica.

In [20]:
import os
import time

# Guardar en ambos formatos
df_limpio.to_csv("/tmp/datos_test.csv", index=False)
df_limpio.to_parquet("/tmp/datos_test.parquet")

# Comparar tamano
size_csv = os.path.getsize("/tmp/datos_test.csv")
size_parquet = os.path.getsize("/tmp/datos_test.parquet")

print(f"CSV:     {size_csv / 1024**2:.2f} MB")
print(f"Parquet: {size_parquet / 1024**2:.2f} MB")
print(f"Parquet es {size_csv / size_parquet:.1f}x mas pequeno")

CSV:     2.85 MB
Parquet: 1.32 MB
Parquet es 2.2x mas pequeno


In [21]:
# Comparar velocidad de lectura
t0 = time.perf_counter()
_ = pd.read_csv("/tmp/datos_test.csv")
t_csv = time.perf_counter() - t0

t0 = time.perf_counter()
_ = pd.read_parquet("/tmp/datos_test.parquet")
t_parquet = time.perf_counter() - t0

print(f"Lectura CSV:     {t_csv:.3f}s")
print(f"Lectura Parquet: {t_parquet:.3f}s")
print(f"Parquet es {t_csv / t_parquet:.1f}x mas rapido")

Lectura CSV:     0.064s
Lectura Parquet: 0.089s
Parquet es 0.7x mas rapido


In [22]:
# El problema real: CSV pierde tipos
df_from_csv = pd.read_csv("/tmp/datos_test.csv")
df_from_parquet = pd.read_parquet("/tmp/datos_test.parquet")

print("Tipos desde CSV (reinferidos):")
print(df_from_csv.dtypes)
print()
print("Tipos desde Parquet (preservados):")
print(df_from_parquet.dtypes)
print()
print("CSV pierde: nullable ints, categories, datetime, boolean.")
print("Parquet preserva todo tal cual lo guardaste.")

Tipos desde CSV (reinferidos):
id_cliente    float64
nombre         object
genero         object
edad            int64
monto         float64
fecha          object
activo         object
region         object
dtype: object

Tipos desde Parquet (preservados):
id_cliente             Int64
nombre        string[python]
genero              category
edad                   Int16
monto                float32
fecha         datetime64[ns]
activo               boolean
region              category
dtype: object

CSV pierde: nullable ints, categories, datetime, boolean.
Parquet preserva todo tal cual lo guardaste.


In [23]:
# Bonus: Parquet permite leer solo columnas especificas
t0 = time.perf_counter()
df_parcial = pd.read_parquet("/tmp/datos_test.parquet", columns=["id_cliente", "monto"])
t_parcial = time.perf_counter() - t0

print(f"Lectura parcial (2 de {len(df_limpio.columns)} cols): {t_parcial:.3f}s")
print(f"Memoria: {df_parcial.memory_usage(deep=True).sum() / 1024**2:.2f} MB")
print("No necesitas cargar 1GB si solo quieres 2 columnas.")

Lectura parcial (2 de 8 cols): 0.010s
Memoria: 1.40 MB
No necesitas cargar 1GB si solo quieres 2 columnas.


---
## 8. Profiling: vectorizacion vs apply vs iterrows

No todos los caminos para la misma operacion son iguales.

In [24]:
# Dataset grande para que las diferencias se noten
n_bench = 500_000
df_bench = pd.DataFrame({
    "precio": np.random.uniform(10, 1000, n_bench),
    "cantidad": np.random.randint(1, 50, n_bench),
    "descuento": np.random.uniform(0, 0.3, n_bench),
})

# Operacion: total = precio * cantidad * (1 - descuento)

In [25]:
# Metodo 1: Vectorizado (el bueno)
t0 = time.perf_counter()
df_bench["total_vec"] = df_bench["precio"] * df_bench["cantidad"] * (1 - df_bench["descuento"])
t_vec = time.perf_counter() - t0
print(f"Vectorizado: {t_vec:.4f}s")

Vectorizado: 0.0044s


In [26]:
# Metodo 2: apply (el que sugieren los LLMs)
t0 = time.perf_counter()
df_bench["total_apply"] = df_bench.apply(
    lambda row: row["precio"] * row["cantidad"] * (1 - row["descuento"]),
    axis=1,
)
t_apply = time.perf_counter() - t0
print(f"apply:       {t_apply:.4f}s  ({t_apply/t_vec:.0f}x mas lento)")

apply:       6.7370s  (1540x mas lento)


In [27]:
# Metodo 3: iterrows (el malo)
t0 = time.perf_counter()
totales = []
for _, row in df_bench.head(50_000).iterrows():  # solo 50k para no esperar
    totales.append(row["precio"] * row["cantidad"] * (1 - row["descuento"]))
t_iter = time.perf_counter() - t0
# Escalar al total
t_iter_est = t_iter * (n_bench / 50_000)
print(f"iterrows:    {t_iter_est:.4f}s estimado ({t_iter_est/t_vec:.0f}x mas lento)")

iterrows:    26.1984s estimado (5990x mas lento)


In [28]:
# Resumen
print("\n=== Resumen de velocidad ===")
print(f"Vectorizado: {t_vec:.4f}s  (baseline)")
print(f"apply:       {t_apply:.4f}s  ({t_apply/t_vec:.0f}x mas lento)")
print(f"iterrows:    {t_iter_est:.4f}s  ({t_iter_est/t_vec:.0f}x mas lento, estimado)")
print()
print("Si un LLM te sugiere .apply(axis=1), preguntate:")
print("¿Hay una operacion vectorizada que haga lo mismo?")
print("En el 80% de los casos la respuesta es si.")


=== Resumen de velocidad ===
Vectorizado: 0.0044s  (baseline)
apply:       6.7370s  (1540x mas lento)
iterrows:    26.1984s  (5990x mas lento, estimado)

Si un LLM te sugiere .apply(axis=1), preguntate:
¿Hay una operacion vectorizada que haga lo mismo?
En el 80% de los casos la respuesta es si.


---
## 9. Ejercicios

### Ejercicio 1: Diagnostico
Usa `df_sucio` y responde:
- ¿Cuanta memoria usa?
- ¿Que columnas son `object` y podrian ser `category`?
- ¿Hay columnas numericas que fueron promovidas a `float` por missings?

### Ejercicio 2: Pipeline
Toma el `df_sucio` y construye tu propio pipeline con `.pipe()` que:
1. Valide que tiene al menos 4 columnas
2. Normalice los nombres de columnas
3. Limpie missings disfrazados en columnas de texto
4. Convierta `fecha` a datetime
5. Exporte a Parquet

Incluye al menos 2 `assert` entre pasos.

### Ejercicio 3: Optimizacion
El siguiente codigo es lento. Reescribelo usando operaciones vectorizadas:

```python
# Codigo lento — reescribir
df["categoria"] = df["monto"].apply(
    lambda x: "alto" if x > 1000 else ("medio" if x > 100 else "bajo")
)
```

In [29]:
# Espacio para resolver ejercicios


---

## Resumen

| Concepto | Clave |
|----------|-------|
| `np.nan` | Es float → promueve int a float, bool a object |
| `pd.NA` + nullable dtypes | El futuro: `Int64`, `Float64`, `boolean`, `string` |
| Missings en texto | Normalizar `""`, `"N/A"`, `"null"`, etc. antes de analizar |
| Memoria | `info(memory_usage='deep')`, `category`, downcast |
| Pipeline | Funciones `df → df`, `.pipe()`, asserts entre pasos |
| Parquet > CSV | Mas rapido, mas pequeno, preserva tipos |
| Vectorizar > apply | 10-100x de diferencia, siempre buscar la alternativa vectorizada |